In [55]:
import pandapower as pp
from pandapower.networks import create_cigre_network_mv
from pandapower.plotting.plotly import pf_res_plotly

net = create_cigre_network_mv(with_der="all")

Transforming the HV network in the CIGRE-MV model to 220kV

In [56]:
# Transforming bus[0] in a 220kV bus
net.bus.at[0, "vn_kv"] = 220.0
# Transforming trafo[0] and trafo[1] in 220 kv --> 20kV busses 
net.trafo.at[0, "vn_hv_kv"] = 220.0
net.trafo.at[1, "vn_hv_kv"] = 220.0
print(f"## Bus 0 specs: \n{net.bus.iloc[0]}")
print(f"## Trafo 0 specs: \n{net.trafo.iloc[0]}")
print(f"## Trafo 1 specs: \n{net.trafo.iloc[1]}")

## Bus 0 specs: 
name                                               Bus 0
vn_kv                                              220.0
type                                                   b
zone                                            CIGRE_MV
in_service                                          True
geo           {"coordinates":[7.0,16.0], "type":"Point"}
Name: 0, dtype: object
## Trafo 0 specs: 
name                       Trafo 0-1
std_type                        None
hv_bus                             0
lv_bus                             1
sn_mva                          25.0
vn_hv_kv                       220.0
vn_lv_kv                        20.0
vk_percent                  12.00107
vkr_percent                     0.16
pfe_kw                           0.0
i0_percent                       0.0
shift_degree                    30.0
tap_side                        None
tap_neutral                      NaN
tap_min                          NaN
tap_max                          NaN
tap_ste

In [57]:
net.sgen

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
0,PV 3,3,0.020,0.0,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
1,PV 4,4,0.020,0.0,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
2,PV 5,5,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
3,PV 6,6,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
4,PV 8,8,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
5,PV 9,9,0.030,0.0,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
6,PV 10,10,0.040,0.0,NaN,NaN,0.040,1.0,False,<NA>,False,NaN,True,PV,True
7,PV 11,11,0.010,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
8,WKA 7,7,1.500,0.0,NaN,NaN,1.500,1.0,False,<NA>,False,NaN,True,WP,True
9,Residential fuel cell 1,5,0.033,0.0,NaN,NaN,0.033,1.0,False,<NA>,False,NaN,True,Residential fuel cell,True


Adding 100kV to each PV generator to match the Italian use-case

In [58]:
for i in range(0, 8):
    if net.sgen.at[i, "p_mw"] < 0.1: 
        net.sgen.at[i, "p_mw"] = net.sgen.at[i, "p_mw"] + 0.1 # Adding 0.1 MW to each PV generator 
        net.sgen.at[i, "q_mvar"] = net.sgen.at[i, "p_mw"] * 0.1
net.sgen

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
0,PV 3,3,0.120,0.012,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
1,PV 4,4,0.120,0.012,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
2,PV 5,5,0.130,0.013,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
3,PV 6,6,0.130,0.013,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
4,PV 8,8,0.130,0.013,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
5,PV 9,9,0.130,0.013,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
6,PV 10,10,0.140,0.014,NaN,NaN,0.040,1.0,False,<NA>,False,NaN,True,PV,True
7,PV 11,11,0.110,0.011,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
8,WKA 7,7,1.500,0.000,NaN,NaN,1.500,1.0,False,<NA>,False,NaN,True,WP,True
9,Residential fuel cell 1,5,0.033,0.000,NaN,NaN,0.033,1.0,False,<NA>,False,NaN,True,Residential fuel cell,True


In [59]:
# Setting load scaling in case needed
for i in range(0, 14):
    net.load.at[i, "scaling"] = 1

In [60]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
0,Line 1-2,CABLE_CIGRE_MV,1,2,2.82,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
1,Line 2-3,CABLE_CIGRE_MV,2,3,4.42,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
2,Line 3-4,CABLE_CIGRE_MV,3,4,0.61,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
3,Line 4-5,CABLE_CIGRE_MV,4,5,0.56,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
4,Line 5-6,CABLE_CIGRE_MV,5,6,1.54,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
5,Line 7-8,CABLE_CIGRE_MV,7,8,1.67,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
6,Line 8-9,CABLE_CIGRE_MV,8,9,0.32,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
7,Line 9-10,CABLE_CIGRE_MV,9,10,0.77,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
8,Line 10-11,CABLE_CIGRE_MV,10,11,0.33,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None
9,Line 3-8,CABLE_CIGRE_MV,3,8,1.30,0.501,0.716,151.17490,0.0,0.145,1.0,1,cs,True,None


In [61]:
pf_res_plotly(net, auto_open=False, aspectratio=(1.2,1))

There are no Power Flow results. A Newton-Raphson power flow will be executed.


In [62]:
# Modifiy the reactive power setpoint for all the PV generators
for elem in net.sgen.iterrows():
    if str.startswith(elem[1]["name"], "PV"):  
        p_mw = net.sgen.at[elem[0], "p_mw"]
        net.sgen.at[elem[0], "q_mvar"] = -p_mw * 0.4 # Setting reactive power to -40% of active power (absorbing reactive power)
net.sgen

# TODO: we could study how the number of violated busses changes with the number of compromised PVs

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
0,PV 3,3,0.120,-0.048,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
1,PV 4,4,0.120,-0.048,NaN,NaN,0.020,1.0,False,<NA>,False,NaN,True,PV,True
2,PV 5,5,0.130,-0.052,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
3,PV 6,6,0.130,-0.052,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
4,PV 8,8,0.130,-0.052,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
5,PV 9,9,0.130,-0.052,NaN,NaN,0.030,1.0,False,<NA>,False,NaN,True,PV,True
6,PV 10,10,0.140,-0.056,NaN,NaN,0.040,1.0,False,<NA>,False,NaN,True,PV,True
7,PV 11,11,0.110,-0.044,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
8,WKA 7,7,1.500,0.000,NaN,NaN,1.500,1.0,False,<NA>,False,NaN,True,WP,True
9,Residential fuel cell 1,5,0.033,0.000,NaN,NaN,0.033,1.0,False,<NA>,False,NaN,True,Residential fuel cell,True


In [63]:
pp.runpp(net)
pf_res_plotly(net, auto_open=False, aspectratio=(1.2,1))